# A)

In [1]:
# A)
import pandas as pd

penguins = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv')

print(penguins)

    species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen            39.1           18.7              181.0   
1    Adelie  Torgersen            39.5           17.4              186.0   
2    Adelie  Torgersen            40.3           18.0              195.0   
3    Adelie  Torgersen             NaN            NaN                NaN   
4    Adelie  Torgersen            36.7           19.3              193.0   
..      ...        ...             ...            ...                ...   
339  Gentoo     Biscoe             NaN            NaN                NaN   
340  Gentoo     Biscoe            46.8           14.3              215.0   
341  Gentoo     Biscoe            50.4           15.7              222.0   
342  Gentoo     Biscoe            45.2           14.8              212.0   
343  Gentoo     Biscoe            49.9           16.1              213.0   

     body_mass_g     sex  
0         3750.0    MALE  
1         3800.0  FEMALE  
2     

In [2]:
gentoo = penguins.loc[penguins['species'] == 'Gentoo'].dropna()
gentoo_females = gentoo.loc[gentoo['sex'] == 'FEMALE']
gentoo_males = gentoo.loc[gentoo['sex'] == 'MALE']


In [3]:
import numpy as np

female_mean = np.mean(gentoo_females['flipper_length_mm'])
male_mean = np.mean(gentoo_males['flipper_length_mm'])

female_var = gentoo_females.var()['flipper_length_mm']
male_var = gentoo_males.var()['flipper_length_mm']

/tmp/ipykernel_85020/3603806829.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  female_var = gentoo_females.var()['flipper_length_mm']
/tmp/ipykernel_85020/3603806829.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  male_var = gentoo_males.var()['flipper_length_mm']


In [4]:
print("MEAN:")
print(f"female: {female_mean}  male: {male_mean}")
print("VARIANCE:")
print(f"female: {female_var} male: {male_var}")

MEAN:
female: 212.70689655172413  male: 221.54098360655738
VARIANCE:
female: 15.19328493647913 male: 32.185792349726775


# B) + C)

In [5]:
from scipy import stats


GENTOO_MALE_PROB = .5
GENTOO_FEMALE_PROB = .5


def bayes_classify(flipper_length, male_mean, female_mean, male_var, female_var):
    female = stats.norm.pdf(flipper_length, female_mean, female_var)
    male = stats.norm.pdf(flipper_length, male_mean, male_var)
    #print(f"With Flipper Length: {flipper_length}")

    f= (.5*female) / ((.5*female)+(.5*male))
    m= (.5*male) / ((.5*male)+(.5*female))

    #print(f'probability to be a female {f}, probability to be a male {m}')
    return (f, m)

def sex_prediction(*probabilities):
    (female_prob, male_prob) = probabilities
    return "FEMALE" if female_prob > male_prob else "MALE"

Bela = (bayes_classify(240, male_mean, female_mean, male_var, female_var))

print(f"Bela's probability of being a female is : {Bela[0]}".format(*Bela))
print(f"Bela's probability of being a male is : {Bela[1]}".format(*Bela))

Bela's probability of being a female is : 0.3321710409656932
Bela's probability of being a male is : 0.6678289590343068


In [6]:
gentoo['predicted_sex_vector'] = gentoo.apply(lambda row : bayes_classify(
    row['flipper_length_mm'], male_mean, female_mean, male_var, female_var),
    axis=1)

gentoo['predicted_sex'] = gentoo.apply(lambda row : sex_prediction(*bayes_classify(
    row['flipper_length_mm'], male_mean, female_mean, male_var, female_var)),
    axis=1)

print(gentoo)


    species  island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
220  Gentoo  Biscoe            46.1           13.2              211.0   
221  Gentoo  Biscoe            50.0           16.3              230.0   
222  Gentoo  Biscoe            48.7           14.1              210.0   
223  Gentoo  Biscoe            50.0           15.2              218.0   
224  Gentoo  Biscoe            47.6           14.5              215.0   
..      ...     ...             ...            ...                ...   
338  Gentoo  Biscoe            47.2           13.7              214.0   
340  Gentoo  Biscoe            46.8           14.3              215.0   
341  Gentoo  Biscoe            50.4           15.7              222.0   
342  Gentoo  Biscoe            45.2           14.8              212.0   
343  Gentoo  Biscoe            49.9           16.1              213.0   

     body_mass_g     sex                       predicted_sex_vector  \
220       4500.0  FEMALE   (0.6895444309386145, 0.31

#D)


In [7]:
import altair as alt

chart = alt.Chart(gentoo).mark_bar().encode(
    x=alt.X('sex:N'),
    y=alt.Y('count(sex):Q')
).properties(title='Gentoo Penguins Count by Sex')

predicted_chart = alt.Chart(gentoo).mark_bar().encode(
    x=alt.X('predicted_sex:N'),
    y=alt.Y('count(predicted_sex):Q')
).properties(title='Gentoo Penguins Count by Predicted Sex')

chart | predicted_chart

alt.HConcatChart(...)

## Conclusions
This shows that through flipper length alone, Gentoo penguins' sex cannot be determined, as the variance for male flipper length is so large that the normal distribution will "predict" the penguin to be female all the time. More accurate calculations can be achieved taking into account body mass, amongst other factors.